In [1]:
# Digit Recognizer

In [18]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
get_ipython().magic('matplotlib inline')

# machine learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

#Evalaluation
from sklearn.cross_validation import cross_val_score
from sklearn import metrics

# Grid
from sklearn.model_selection import GridSearchCV

In [3]:
data = pd.read_csv("train.csv")
kaggletest = pd.read_csv("test.csv")

In [4]:
X = data.iloc[:,1:]
y = data.iloc[:,0]

In [5]:
X[X>0] = 1

D:\WindowsApps\Anaconda\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
D:\WindowsApps\Anaconda\envs\tensorflow\lib\site-packages\pandas\core\frame.py:2464: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._where(-key, value, inplace=True)


In [6]:
from sklearn.model_selection import train_test_split

# use train/test split with different random_state values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=4)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((31500, 784), (31500,), (10500, 784), (10500,))

# MODELING

In [7]:
from sklearn.cross_validation import cross_val_score
from sklearn import metrics
from sklearn import datasets

D:\WindowsApps\Anaconda\envs\tensorflow\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
# Linear SVM
from sklearn import svm

# Define Model
model = svm.LinearSVC()
#Fit Model
print(model.fit(X_train, y_train))

y_pred = model.predict(X_test)
#print(model.score(y_test, y_pred))
metrics.accuracy_score(y_test, y_pred)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)


0.89895238095238095

In [9]:
# Logistic
from sklearn.linear_model import LogisticRegression
model= LogisticRegression()
#Fit Model
print(model.fit(X_train, y_train))

y_pred = model.predict(X_test)
#print(model.score(y_test, y_pred))
#Fit Model
scores= cross_val_score(model, X, y, cv=10, scoring='accuracy')
print(scores.mean()*100)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
90.7546966431


0.90571428571428569

In [10]:
# Linear SVM
from sklearn import svm

# Define Model
model = svm.LinearSVC()
#Fit Model
scores= cross_val_score(model, X, y, cv=2, scoring='accuracy')
scores.mean()*100

89.28093583566536

In [16]:
%%time
%%memit
# Tree
from sklearn.tree import DecisionTreeClassifier

# Define Model
model = DecisionTreeClassifier(splitter="random")
#Fit Model
scores= cross_val_score(model, X, y, cv=4, scoring='accuracy')
print(scores.mean())

0.864642668017
peak memory: 1054.99 MiB, increment: 345.53 MiB
Wall time: 12.8 s


In [17]:
%%time
%%memit
random_forest = RandomForestClassifier(n_estimators=500)
random_forest.fit(X_train, y_train)
Y_pred = random_forest.predict(X_test)
print(random_forest.score(X_train, y_train))
acc_random_forest = round(random_forest.score(X_train, y_train) * 100, 2)
print(acc_random_forest)
print(cross_val_score(random_forest, X, y, cv=10, scoring='accuracy').mean()*100)

NameError: name 'RandomForestClassifier' is not defined

In [ ]:
%%time
%%memit
tree = DecisionTreeClassifier()
bag = BaggingClassifier(tree, n_estimators=100, max_samples=0.8)

print(cross_val_score(bag, X, y, cv=10, scoring='accuracy').mean()*100)

In [12]:
# Logistic
from sklearn.linear_model import LogisticRegression

model= LogisticRegression()

#Fit Model
scores= cross_val_score(model, X, y, cv=4, scoring='accuracy')
scores.mean()

0.90502369007384842

In [15]:
# Submit
model= LogisticRegression()
#Fit Model
model.fit(X, y)

submission = model.predict(kaggletest)

In [18]:
len(submission)

28000

In [22]:
submission[:10]
df = pd.DataFrame(submission)
df.index.name='ImageId'
df.index+=1
df.columns=['Label']
df.to_csv('results.csv',header=True)

## Grid Optimization

In [33]:
from sklearn.model_selection import GridSearchCV
# create a parameter grid: map the parameter names to the values that should be searched
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]
print(param_grid)

# instantiate the grid
model = svm.LinearSVC()
grid = GridSearchCV(model, param_grid, cv=10, scoring='accuracy')


[{'kernel': ['linear'], 'C': [1, 10, 100, 1000]}, {'kernel': ['rbf'], 'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001]}]


In [23]:
# grid.fit(X, y)

In [ ]:
print(grid.best_params_)
print(grid.best_score_)
print(grid.best_estimator_)

In [ ]:
# search for an optimal value of G for SVM
G = [0.1, 0.01, 0.001]
G_scores = []
G_test_scores = []
for k in k_range:
    model = svm.SVC(gamma=G, C=100)
    model.fit(X_train, y_train)
    y_test_pred = knn.predict(X_test)
    k_test_scores.append(metrics.accuracy_score(y_test, y_test_pred))
    

df = pd.DataFrame(list(k_scores),list(k_test_scores), columns=["1","2"])
df

In [20]:
MYG = list(range(1, 30))
print(MYG)
G = [0.1, 0.01, 0.001]
print(G)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
[0.1, 0.01, 0.001]


In [ ]:
# search for an optimal value of K for KNN
k_range = list(range(1, 31))
k_scores = []
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X, y, cv=10, scoring='accuracy')
    k_scores.append(scores.mean())
df = pd.DataFrame(k_scores, index=k_range, columns=["Scores"])
print(df)
df['Scores'].argmax()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# plot the value of K for KNN (x-axis) versus the cross-validated accuracy (y-axis)
plt.plot(k_range, k_scores)
plt.xlabel('Hyper Parameter')
plt.ylabel('Cross-Validated Accuracy')

In [27]:
from sklearn.linear_model import LogisticRegression

model= LogisticRegression()
model.fit(X_train, y_train)
model.score(y_test, y_pred)

/Applications/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


ValueError: X has 500 features per sample; expecting 784